In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e23/sample_submission.csv
/kaggle/input/playground-series-s3e23/train.csv
/kaggle/input/playground-series-s3e23/test.csv


In [2]:
import matplotlib.pyplot as plt
SEED=9

In [3]:
df = pd.read_csv('/kaggle/input/playground-series-s3e23/train.csv')

In [4]:
df.head()

,id,loc,v(g),ev(g),iv(g),n,v,l,d,i,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,0,22.0,3.0,1.0,2.0,60.0,278.63,0.06,19.56,14.25,...,17,1,1,0,16.0,9.0,38.0,22.0,5.0,False
1,1,14.0,2.0,1.0,2.0,32.0,151.27,0.14,7.00,21.11,...,11,0,1,0,11.0,11.0,18.0,14.0,3.0,False
2,2,11.0,2.0,1.0,2.0,45.0,197.65,0.11,8.05,22.76,...,8,0,1,0,12.0,11.0,28.0,17.0,3.0,False
3,3,8.0,1.0,1.0,1.0,23.0,94.01,0.19,5.25,17.86,...,4,0,2,0,8.0,6.0,16.0,7.0,1.0,True
4,4,11.0,2.0,1.0,2.0,17.0,60.94,0.18,5.63,12.44,...,7,0,2,0,7.0,6.0,10.0,10.0,3.0,False


In [5]:
df.shape

(101763, 23)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101763 entries, 0 to 101762
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 101763 non-null  int64  
 1   loc                101763 non-null  float64
 2   v(g)               101763 non-null  float64
 3   ev(g)              101763 non-null  float64
 4   iv(g)              101763 non-null  float64
 5   n                  101763 non-null  float64
 6   v                  101763 non-null  float64
 7   l                  101763 non-null  float64
 8   d                  101763 non-null  float64
 9   i                  101763 non-null  float64
 10  e                  101763 non-null  float64
 11  b                  101763 non-null  float64
 12  t                  101763 non-null  float64
 13  lOCode             101763 non-null  int64  
 14  lOComment          101763 non-null  int64  
 15  lOBlank            101763 non-null  int64  
 16  lo

In [7]:
df.drop('id',axis=1,inplace=True)

In [8]:
df['defects'] = df['defects'].apply(lambda x: 1 if x==True else 0)

In [9]:
df['defects'].value_counts()

defects
0    78699
1    23064
Name: count, dtype: int64

In [10]:
def remove_outliers(df, columns, factor=1.5):
    for column in columns:
        Q1 = df[column].quantile(0.05)
        Q3 = df[column].quantile(0.95)
        IQR = Q3 - Q1
        lower_bound = Q1 - factor * IQR
        upper_bound = Q3 + factor * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

In [11]:
df_cols = df.columns.to_list()
df_cols.pop(-1)

'defects'

In [12]:
#df = remove_outliers(df,df_cols,factor=1.5)

In [13]:
df.shape

(101763, 22)

In [14]:
y = df.iloc[:,-1].values
X = df.iloc[:,:-1]

In [15]:
X.shape

(101763, 21)

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,stratify=y,random_state=SEED)

# XGBoost

In [17]:
# params found with random search
params = {'subsample': 0.6, 'reg_lambda': 0.2, 'reg_alpha': 1.0, 'n_estimators': 750, 'min_child_weight': 4, 'max_depth': 3, 'learning_rate': 0.03, 'gamma': 0.6, 'colsample_bytree': 0.6}

In [18]:
import xgboost as xgb
#tree_method='gpu_hist' for GPU
xgb_classifier = xgb.XGBClassifier(objective='binary:logistic',eval_metric='auc',random_state=SEED,**params)

In [19]:
xgb_classifier.fit(X,y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=0.6, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.03, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=4, missing=nan, monotone_constraints=None,
              n_estimators=750, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=9, ...)

In [44]:
accuracy_score(xgb_classifier.predict(X_test),y_test)

0.8278692837972952

In [35]:
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    'n_estimators': np.arange(500, 1000, 50),  # Values from 500 to 900 in steps of 50
    'learning_rate': np.linspace(0.01, 0.1, num=10),  # Values from 0.01 to 0.1, 10 evenly spaced values
    'max_depth': np.arange(1, 11),  # Values from 1 to 10
    'min_child_weight': np.arange(1, 6),  # Values from 1 to 5
    'subsample': np.arange(0.5, 1.0, 0.1),  # Values from 0.5 to 0.9 in steps of 0.1
    'colsample_bytree': np.arange(0.5, 1.0, 0.1),  # Values from 0.5 to 0.9 in steps of 0.1
    'gamma': np.linspace(0, 1, num=11),  # Values from 0 to 1, 11 evenly spaced values
    'reg_alpha': np.linspace(0, 1, num=11),  # Values from 0 to 1, 11 evenly spaced values
    'reg_lambda': np.linspace(0, 1, num=11)  # Values from 0 to 1, 11 evenly spaced values
}

In [36]:
xgb_random_search = RandomizedSearchCV(xgb_classifier, param_distributions=param_dist, n_iter=200, scoring='roc_auc', n_jobs=-1, cv=5, verbose=3, random_state=SEED)

In [37]:
xgb_random_search.fit(X_train, y_train)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[CV 3/5] END colsample_bytree=0.6, gamma=0.30000000000000004, learning_rate=0.08, max_depth=3, min_child_weight=5, n_estimators=850, reg_alpha=0.4, reg_lambda=0.30000000000000004, subsample=0.5;, score=0.763 total time=   8.7s
[CV 1/5] END colsample_bytree=0.5, gamma=0.2, learning_rate=0.07, max_depth=7, min_child_weight=1, n_estimators=850, reg_alpha=1.0, reg_lambda=0.8, subsample=0.6;, score=0.742 total time=  16.3s
[CV 5/5] END colsample_bytree=0.5, gamma=0.2, learning_rate=0.07, max_depth=7, min_child_weight=1, n_estimators=850, reg_alpha=1.0, reg_lambda=0.8, subsample=0.6;, score=0.743 total time=  12.6s
[CV 4/5] END colsample_bytree=0.6, gamma=0.1, learning_rate=0.06000000000000001, max_depth=4, min_child_weight=4, n_estimators=550, reg_alpha=1.0, reg_lambda=0.0, subsample=0.5;, score=0.773 total time=   6.3s
[CV 3/5] END colsample_bytree=0.5, gamma=0.5, learning_rate=0.030000000000000006, max_depth=9, min_child_weight=4, n_estimators=950, reg_alpha=0.30000000000000004, reg_lambd

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='auc',
                                           feature_types=None, gamma=None,
                                           gpu_id=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rat...
                                        'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                                        'min_child_weight': array([1, 2, 3, 4, 5]),
                                        'n_estimators': array([500, 550, 600, 650, 700, 750, 800, 850, 900, 950]),
                                        'reg_alpha': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
                                        'reg_lambda': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
                                        'subsample': array([0.5, 0.6, 0.7, 0.8, 0.9])},
                   random_state=9, scoring='roc_auc', verbose=3)

In [38]:
print(xgb_random_search.best_params_)
print(xgb_random_search.best_score_)

{'subsample': 0.6, 'reg_lambda': 0.2, 'reg_alpha': 1.0, 'n_estimators': 750, 'min_child_weight': 4, 'max_depth': 3, 'learning_rate': 0.030000000000000006, 'gamma': 0.6000000000000001, 'colsample_bytree': 0.6}
0.7703550217756754


In [39]:
best_model = xgb_random_search.best_estimator_
xgb_preds = best_model.predict(X_test)
print(accuracy_score(y_test, xgb_preds))

0.8243435247066253

[CV 4/5] END colsample_bytree=0.8999999999999999, gamma=0.0, learning_rate=0.07, max_depth=2, min_child_weight=3, n_estimators=700, reg_alpha=1.0, reg_lambda=0.30000000000000004, subsample=0.6;, score=0.775 total time=   4.5s
[CV 3/5] END colsample_bytree=0.8999999999999999, gamma=0.9, learning_rate=0.04000000000000001, max_depth=3, min_child_weight=2, n_estimators=600, reg_alpha=0.6000000000000001, reg_lambda=0.1, subsample=0.5;, score=0.766 total time=   5.1s
[CV 2/5] END colsample_bytree=0.7, gamma=0.1, learning_rate=0.01, max_depth=2, min_child_weight=1, n_estimators=750, reg_alpha=0.8, reg_lambda=0.9, subsample=0.7999999999999999;, score=0.767 total time=   5.2s
[CV 3/5] END colsample_bytree=0.7, gamma=0.1, learning_rate=0.01, max_depth=2, min_child_weight=1, n_estimators=750, reg_alpha=0.8, reg_lambda=0.9, subsample=0.7999999999999999;, score=0.767 total time=   5.1s
[CV 5/5] END colsample_bytree=0.7999999999999999, gamma=0.30000000000000004, learning_rate=0.0

# TF-DF

In [20]:
!pip install tensorflow tensorflow_decision_forests

In [21]:
import tensorflow as tf

In [22]:
np.random.seed=SEED
def split_dataset(dataset, test_ratio=0.20):
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]

train_ds_pd, test_ds_pd = split_dataset(df)
print("{} examples in training, {} examples in testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

81482 examples in training, 20281 examples in testing.


In [23]:
import tensorflow_decision_forests as tfdf

train_data = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label='defects')
test_data = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label='defects')
data=tfdf.keras.pd_dataframe_to_tf_dataset(df, label='defects')

In [21]:
tfdf.keras.get_all_models()

[tensorflow_decision_forests.keras.RandomForestModel,
 tensorflow_decision_forests.keras.GradientBoostedTreesModel,
 tensorflow_decision_forests.keras.CartModel,
 tensorflow_decision_forests.keras.DistributedGradientBoostedTreesModel]

In [24]:
#tf_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(df, label="defects")

tf_df = tfdf.keras.GradientBoostedTreesModel(task=tfdf.keras.Task.CLASSIFICATION,)

tf_df.compile(metrics=[tf.keras.metrics.AUC(curve="ROC")])
tf_df.fit(train_data)

#print(tf_df.summary())

Use /tmp/tmp8blxudei as temporary training directory
Reading training dataset...


[WARNING 23-10-22 14:10:42.4526 UTC gradient_boosted_trees.cc:1818] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-10-22 14:10:42.4528 UTC gradient_boosted_trees.cc:1829] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-10-22 14:10:42.4529 UTC gradient_boosted_trees.cc:1843] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
Exception ignored in: <function _xla_gc_callback at 0x7f4b2e3243a0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/jax/_src/lib/__init__.py", line 103, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


Training dataset read in 0:00:08.646342. Found 81299 examples.
Training model...
Model trained in 0:00:04.685150
Compiling model...


[INFO 23-10-22 14:10:55.8240 UTC kernel.cc:1243] Loading model from path /tmp/tmp8blxudei/model/ with prefix 9a3689af5d484a86
[INFO 23-10-22 14:10:55.8347 UTC abstract_model.cc:1311] Engine "GradientBoostedTreesQuickScorerExtended" built
[INFO 23-10-22 14:10:55.8355 UTC kernel.cc:1075] Use fast generic engine


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.


In [25]:
predictions = tf_df.predict(test_data)
y_true      = test_ds_pd["defects"]

from sklearn.metrics import roc_auc_score
ROC_AUC = roc_auc_score(y_true, predictions)
print("The ROC AUC score is %.5f" % ROC_AUC )
# after removing the outliers: The ROC AUC score was 0.76545, 'shit'

21/21 [==============================] - 0s 15ms/step
The ROC AUC score is 0.78899


In [26]:
tf_df.evaluate(test_data)

21/21 [==============================] - 1s 16ms/step - loss: 0.0000e+00 - auc: 0.7885


[0.0, 0.7885320782661438]

In [24]:
tuner = tfdf.tuner.RandomSearch(num_trials=50)

tuner.choice("min_examples", [2, 5, 6,7,8,9, 10])
tuner.choice("categorical_algorithm", ["CART", "RANDOM"])

local_search_space = tuner.choice("growing_strategy", ["LOCAL"])
local_search_space.choice("max_depth", [3, 4, 5, 6, 8])

global_search_space = tuner.choice("growing_strategy", ["BEST_FIRST_GLOBAL"], merge=True)
global_search_space.choice("max_num_nodes", [16, 32, 64, 128, 256])

tuner.choice("use_hessian_gain", [True, False])
tuner.choice("shrinkage", [0.02, 0.05, 0.10, 0.15])
tuner.choice("num_candidate_attributes_ratio", [0.2, 0.5, 0.9, 1.0])

In [72]:
tfdf_params = {'min_examples':5, 'categorical_algorithm':'RANDOM', 'growing_strategy':'BEST_FIRST_GLOBAL', 'max_num_nodes':32, 'use_hessian_gain':True ,'shrinkage':0.02 ,'num_candidate_attributes_ratio':1}

In [25]:
tfdf_model = tfdf.keras.GradientBoostedTreesModel(tuner=tuner)
tfdf_model.fit(train_data)

print(tfdf_model.summary())

Use /tmp/tmp30qq8ata as temporary training directory
Reading training dataset...


[WARNING 23-10-22 14:12:35.0089 UTC gradient_boosted_trees.cc:1818] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-10-22 14:12:35.0092 UTC gradient_boosted_trees.cc:1829] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-10-22 14:12:35.0092 UTC gradient_boosted_trees.cc:1843] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".


Training dataset read in 0:00:08.257507. Found 81482 examples.
Training model...
Model trained in 0:11:08.004252
Compiling model...


[INFO 23-10-22 14:23:51.1483 UTC kernel.cc:1243] Loading model from path /tmp/tmp30qq8ata/model/ with prefix 9765a094fd7b4b1b
[INFO 23-10-22 14:23:51.2951 UTC decision_forest.cc:660] Model loaded with 180 root(s), 36792 node(s), and 21 input feature(s).
[INFO 23-10-22 14:23:51.2957 UTC abstract_model.cc:1311] Engine "GradientBoostedTreesOptPred" built
[INFO 23-10-22 14:23:51.2957 UTC kernel.cc:1075] Use fast generic engine


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.
Model: "gradient_boosted_trees_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (21):
	b
	branchCount
	d
	e
	ev(g)
	i
	iv(g)
	l
	lOBlank
	lOCode
	lOComment
	loc
	locCodeAndComment
	n
	t
	total_Op
	total_Opnd
	uniq_Op
	uniq_Opnd
	v
	v(g)

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.               "loc"  0.272470 ################
    2.       "branchCount"  0.169109 ####
    3.   

In [26]:
tfdf_predictions = tfdf_model.predict(test_data)
y_true = test_ds_pd["defects"]

from sklearn.metrics import roc_auc_score
ROC_AUC = roc_auc_score(y_true, tfdf_predictions)
print("The ROC AUC score is %.5f" % ROC_AUC )

21/21 [==============================] - 0s 15ms/step
The ROC AUC score is 0.78873


In [28]:
tfdf_model.fit(data)

Reading training dataset...
Training dataset read in 0:00:01.541386. Found 101763 examples.
Training model...
Model trained in 0:13:32.028553
Compiling model...


[INFO 23-10-21 09:36:56.7027 UTC kernel.cc:1243] Loading model from path /tmp/tmpor3r57w9/model/ with prefix 833a4fcfc7604493
[INFO 23-10-21 09:36:56.8110 UTC abstract_model.cc:1311] Engine "GradientBoostedTreesQuickScorerExtended" built
[INFO 23-10-21 09:36:56.8110 UTC kernel.cc:1075] Use fast generic engine


Model compiled.


# Submission

In [27]:
test = pd.read_csv('/kaggle/input/playground-series-s3e23/test.csv')
test_id = test.id
test.drop('id',axis=1,inplace=True)

In [28]:
test_tf = tfdf.keras.pd_dataframe_to_tf_dataset(test)

In [29]:
xgb_probas = xgb_classifier.predict_proba(test)

In [30]:
tfdf_probas = tfdf_model.predict(test_tf)
tfdf_probas=tfdf_probas.reshape(-1)

68/68 [==============================] - 1s 15ms/step


In [38]:
#20 3
preds = (xgb_probas[:,1]+tfdf_probas)/2

In [79]:
ROC_AUC_best = 0 

for i in range(15,21):
    for j in range(2,5):
        predictions = (xgb_classifier.predict_proba(test_ds_pd.drop('defects',axis=1).values)[:,1]*i+tfdf_model.predict(test_data).reshape(-1))*j/(i+j)
        ROC_AUC = roc_auc_score(y_true, predictions)
        if ROC_AUC > ROC_AUC_best:
            ROC_AUC_best = ROC_AUC
            print("The ROC AUC score is %.5f" % ROC_AUC )
            print(i,j,k)

638/638 [==============================] - 1s 1ms/step
The ROC AUC score is 0.80603
15 2 3
638/638 [==============================] - 1s 1ms/step
The ROC AUC score is 0.80603
15 2 4
638/638 [==============================] - 1s 1ms/step
The ROC AUC score is 0.80603
15 3 3
638/638 [==============================] - 1s 1ms/step
The ROC AUC score is 0.80604
15 3 4
638/638 [==============================] - 1s 1ms/step
The ROC AUC score is 0.80604
15 4 3
638/638 [==============================] - 1s 1ms/step
The ROC AUC score is 0.80604
15 4 4
638/638 [==============================] - 1s 1ms/step
The ROC AUC score is 0.80604
16 2 3
638/638 [==============================] - 1s 1ms/step
The ROC AUC score is 0.80604
16 2 4
638/638 [==============================] - 1s 1ms/step
The ROC AUC score is 0.80605
16 3 3
638/638 [==============================] - 1s 1ms/step
The ROC AUC score is 0.80605
16 3 4
638/638 [==============================] - 1s 1ms/step
The ROC AUC score is 0.80605
16 4 3

In [66]:
predictions = (xgb_classifier.predict_proba(test_ds_pd.drop('defects',axis=1).values)[:,1]+tfdf_model.predict(test_data).reshape(-1))/2

from sklearn.metrics import roc_auc_score
ROC_AUC = roc_auc_score(y_true, predictions)
print("The ROC AUC score is %.5f" % ROC_AUC )

19/19 [==============================] - 0s 12ms/step
The ROC AUC score is 0.77705


In [51]:
my_submissions = pd.DataFrame({'id':test_id,'defects':preds})

In [52]:
my_submissions.to_csv('/kaggle/working/submission.csv', index=False)
my_submissions.head()

,id,defects
0,101763,0.225008
1,101764,0.195987
2,101765,0.605131
3,101766,0.458347
4,101767,0.145427


In [53]:
import os
os.chdir(r'../working')
from IPython.display import FileLink
FileLink(r'submission.csv')

/kaggle/working/submission.csv